In [1]:
%load_ext autoreload

In [2]:
import numpy as np
import os
import mcubes
import meshcat
import pydrake
from pydrake.geometry import SceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.plant import MultibodyPlant, AddMultibodyPlantSceneGraph
from pydrake.multibody.parsing import Parser, LoadModelDirectives, ProcessModelDirectives
from pydrake.multibody.tree import RevoluteJoint
from pydrake.all import ConnectMeshcatVisualizer, InverseKinematics, RigidTransform, RotationMatrix
from pydrake.all import BsplineTrajectoryThroughUnionOfHPolyhedra, IrisInConfigurationSpace, IrisOptions, Rgba
import time
from meshcat import Visualizer
from functools import partial
from pydrake.solvers.mosek import MosekSolver


import ipywidgets as widgets
from IPython.display import display

from pydrake.all import MeshcatVisualizerCpp, MeshcatVisualizerParams, Role

from symbolic_parsing_helpers import generate_rationalize_trig_expr_rules
from symbolic_parsing_helpers import xreplace
from symbolic_parsing_helpers import NotRationalFunctionException
import symbolic_parsing_helpers as symHelpers

# from symbolic_parsing_helpers import rationalize_trig_expr
from pydrake.all import RationalFunction


from pydrake.all import (
    ConvexSet, HPolyhedron, Hyperellipsoid,
    MathematicalProgram, Solve, le, IpoptSolver,
    Role, Sphere,VPolytope,
    Iris, IrisOptions, MakeIrisObstacles, Variable
)
from pydrake.all import (
    eq, SnoptSolver,
    Sphere, Ellipsoid, GeometrySet,
    RigidBody_, AutoDiffXd, initializeAutoDiff,
)

import pydrake.symbolic as sym

from meshcat import Visualizer

# Setup meshcat
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=[])

# Sporadically need to run `pkill -f meshcat`

from symbolic_parsing_helpers import generate_rationalize_trig_expr_rules
from symbolic_parsing_helpers import xreplace
from symbolic_parsing_helpers import NotRationalFunctionException
# from symbolic_parsing_helpers import rationalize_trig_expr
from pydrake.all import RationalFunction

In [3]:
# Setup meshcat
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=[])
proc2, zmq_url2, web_url2 = start_zmq_server_as_subprocess(server_args=[])

# Build plant and simulation

In [4]:
#settings
q0 = [0.0, 0.0, 0.0]
q_low = [-1.7,-1.7, -2.0]
q_high = [1.7, 1.7, 2.0]



#marching cubes
q_low_mc = q_low.copy()
q_high_mc =  q_high.copy()
N = 70

In [5]:
vis = Visualizer(zmq_url=zmq_url)
vis.delete()
vis2 = Visualizer(zmq_url=zmq_url2)
vis2.delete()

builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
parser = Parser(plant)
tworob_asset = FindResourceOrThrow("drake/sandbox/assets/doublerob.urdf")
#two_dof_asset = FindResourceOrThrow("drake/sandbox/assets/planar2dof.urdf")
box_asset = FindResourceOrThrow("drake/sandbox/assets/box.urdf")

models =[]
models.append(parser.AddModelFromFile(tworob_asset))
#models.append(parser.AddModelFromFile(one_dof_asset))
models.append(parser.AddModelFromFile(box_asset))

idx = 0
for model in models:
    for joint_index in plant.GetJointIndices(model):
        joint = plant.get_mutable_joint(joint_index)
        if isinstance(joint, RevoluteJoint):
            joint.set_default_angle(q0[idx])
            idx += 1
            
locs = [[0.,0.,0.],[0.,0.,0.]]
idx = 0
for model in models:
    plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("base", model), RigidTransform(locs[idx]))
    idx+=1

plant.Finalize()

visualizer = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url, delete_prefix_on_load=False, )

diagram = builder.Build()
visualizer.load()
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(context)
diagram.Publish(context)

joints = []
idx = 0
for model in models:
    jointindx = plant.GetJointIndices(model)
    for j in jointindx:
        joint = plant.get_mutable_joint(j)
        if isinstance(joint, RevoluteJoint):
            joints.append(joint)
            joints[-1].set_position_limits(lower_limits= np.array([q_low[idx]]), upper_limits= np.array([np.array([q_high[idx]])]))
            idx +=1
        
    
def set_joint_ang(val, idx):
    joints[idx].set_angle(plant_context, val)
    
def set_joint_angles(vals):
    joints[0].set_angle(plant_context, vals[0])
    joints[1].set_angle(plant_context, vals[1])
    joints[2].set_angle(plant_context, vals[2])
    
ik = InverseKinematics(plant, plant_context)
collision_constraint = ik.AddMinimumDistanceConstraint(0.001, 0.01)


def eval_cons(q0, q1, q2, c, tol):
        return 1-1*float(c.evaluator().CheckSatisfied([q0, q1, q2], tol))
    
col_func_handle = partial(eval_cons, c=collision_constraint, tol=0.01)


def eval_cons_rational(t0, t1, t2, c, tol):
        q = convert_t_to_q(np.array([t0, t1, t2])) 
        return 1-1*float(c.evaluator().CheckSatisfied(q, tol))

col_func_handle_rational = partial(eval_cons_rational, c=collision_constraint, tol=0.01)


def showres(q):
    set_joint_ang(q[0],0)
    set_joint_ang(q[1],1)
    set_joint_ang(q[2],2)
    col = col_func_handle(*q)
    t = convert_q_to_t(q)
    if col:
        vis2["q"].set_object(
                meshcat.geometry.Sphere(0.1), meshcat.geometry.MeshLambertMaterial(color=0xFFB900))
        vis2["q"].set_transform(
                meshcat.transformations.translation_matrix(t))
    else:
        vis2["q"].set_object(
                meshcat.geometry.Sphere(0.1), meshcat.geometry.MeshLambertMaterial(color=0x3EFF00))
        vis2["q"].set_transform(
                meshcat.transformations.translation_matrix(t))
    diagram.Publish(context)
    print("              ", end = "\r")
    print(t , end = "\r")

sliders = []
sliders.append(widgets.FloatSlider(min=q_low[0], max=q_high[0], value=0, description='q0'))
sliders.append(widgets.FloatSlider(min=q_low[1], max=q_high[1], value=0, description='q1'))
sliders.append(widgets.FloatSlider(min=q_low[2], max=q_high[2], value=0, description='q2'))

q = q0.copy()
def handle_slider_change(change, idx):
    q[idx] = change['new']
    #print(q, end="\r")
    showres(q)
    
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1


You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7002/static/
Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6001...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/
Connected to meshcat-server.


# Construct Rational Kinematics

In [6]:
from pydrake.all import RationalForwardKinematics
forward_kin = RationalForwardKinematics(plant)
t = forward_kin.t()
q_star = np.zeros(forward_kin.t().shape[0])
def convert_RationalForwardPoseList_to_TransformExpressionList(pose_list):
    ret = []
    for p in pose_list:
        ret.append(p.asRigidTransformExpr())
    return ret

def convert_t_to_q(t):
    q =np.arctan2(2*t/(1+t**2), (1-t**2)/(1+t**2))
    return q


def convert_q_to_t(q):
    t = np.sin(q)/(1+np.cos(q))
    return t

def convert_q_to_t2(q):
    return np.tan(np.divide(q,2))

t_low = convert_q_to_t(q_low)
t_high = convert_q_to_t(q_high)

t_low2 = convert_q_to_t2(q_low)
t_high2 = convert_q_to_t2(q_high)

print(t_low-t_low2)
print(t_high-t_high2)

[0.00000000e+00 0.00000000e+00 2.22044605e-16]
[ 0.00000000e+00  0.00000000e+00 -2.22044605e-16]


In [7]:
#marching cubes

vertices, triangles = mcubes.marching_cubes_func(tuple(t_low), tuple(t_high), N, N, N, col_func_handle_rational, 0.5)
vis2["collision_constraint"].set_object(
            meshcat.geometry.TriangularMeshGeometry(vertices, triangles),
            meshcat.geometry.MeshLambertMaterial(color=0xff0000, wireframe=True))
q = q0.copy()
showres(q)

In [8]:
for slider in sliders:
    display(slider)

display(vis.jupyter_cell())
display(vis2.jupyter_cell())

FloatSlider(value=0.0, description='q0', max=1.7, min=-1.7)

FloatSlider(value=0.0, description='q1', max=1.7, min=-1.7)

FloatSlider(value=0.0, description='q2', max=2.0, min=-2.0)

# IRIS IN T-SPACE CODE

In [9]:
#  Now IRIS in configuration space, using dReal to solve for the growth volume
# through the nonconvex kinematics.

from pydrake.all import (
    eq, SnoptSolver,
    Sphere, Ellipsoid, GeometrySet,
    RigidBody_, AutoDiffXd, initializeAutoDiff,
)
def MakeFromHPolyhedronOrEllipseSceneGraph(query, geom, expressed_in=None):
    shape = query.inspector().GetShape(geom)
    if isinstance(shape, (Sphere, Ellipsoid)):
        return Hyperellipsoid(query, geom, expressed_in)
    return HPolyhedron(query, geom, expressed_in)
def MakeFromVPolytopeOrEllipseSceneGraph(query, geom, expressed_in=None):
    shape = query.inspector().GetShape(geom)
    if isinstance(shape, (Sphere, Ellipsoid)):
        return Hyperellipsoid(query, geom, expressed_in)
    return VPolytope(query, geom, expressed_in)


def TangentPlane(self, point):
    a = 2 * self.A().T @ self.A() @ (point - self.center())
    a = a / np.linalg.norm(a)
    b = a.dot(point)
    return a, b

Hyperellipsoid.TangentPlane = TangentPlane


query = scene_graph.get_query_output_port().Eval(scene_graph.GetMyContextFromRoot(context))

sym_plant = plant.ToSymbolic()
sym_context = sym_plant.CreateDefaultContext()

# For SNOPT test.
snopt = SnoptSolver()

def MakeFromSceneGraph(query, geom, expressed_in=None):
    shape = query.inspector().GetShape(geom)
    if isinstance(shape, (Sphere, Ellipsoid)):
        return Hyperellipsoid(query, geom, expressed_in)
    return HPolyhedron(query, geom, expressed_in)

dReal_polytope_tol = .1

def CheckVolume(E, bodyA, bodyB, setA, setB, A, b, volume):
    prog = MathematicalProgram()
    q = prog.NewContinuousVariables(plant.num_positions(), "q")
    prog.AddBoundingBoxConstraint(lb, ub, q)
    prog.AddLinearConstraint(A, b-np.inf, b, q)
    p_AA =  prog.NewContinuousVariables(3, "p_AA")
    p_BB = prog.NewContinuousVariables(3, "p_BB")
    if volume < np.inf:
        prog.AddConstraint((q-E.center()).T @ E.A().T @ E.A() @ (q-E.center()) <= volume)
    setA.AddPointInSetConstraints(prog, p_AA)
    setB.AddPointInSetConstraints(prog, p_BB)

    sym_plant.SetPositions(sym_context, q)
    X_WA = sym_plant.EvalBodyPoseInWorld(sym_context, bodyA)
    X_WB = sym_plant.EvalBodyPoseInWorld(sym_context, bodyB)
    # Add +0 pending https://github.com/RobotLocomotion/drake/issues/15216
    print(type(X_WA))
    p_WA = X_WA.multiply(p_AA+0)
    p_WB = X_WB.multiply(p_BB+0)
    prog.AddConstraint(eq(p_WA, p_WB))
    prog.SetSolverOption(dReal.id(), "precision", .9*dReal_polytope_tol)
    result = dReal.Solve(prog)
def GrowthVolumeRational(E, X_WA, X_WB, setA, setB, A, b, guess=None):
    snopt = SnoptSolver()
    prog = MathematicalProgram()
    t = forward_kin.t()
    prog.AddDecisionVariables(t)
    
    if guess is not None:
        prog.SetInitialGuess(t, guess)

    prog.AddLinearConstraint(A, b-np.inf, b, t)
    p_AA = prog.NewContinuousVariables(3, "p_AA")
    p_BB = prog.NewContinuousVariables(3, "p_BB")
    setA.AddPointInSetConstraints(prog, p_AA)
    setB.AddPointInSetConstraints(prog, p_BB)
    prog.AddQuadraticErrorCost(E.A().T @ E.A(), E.center(), t)

    p_WA = X_WA.multiply(p_AA+0)

    p_WB = X_WB.multiply(p_BB+0)

    prog.AddConstraint(eq(p_WA, p_WB))
    result = snopt.Solve(prog)
    return result.is_success(), result.get_optimal_cost(), result.GetSolution(t)


In [10]:
def iris_rational_space(query, point, require_containment_points=[], termination_threshold=2e-2, iteration_limit=100):
    dReal_polytope_tol = .1
    ellipsoid_epsilon = 1e-1
    dim = plant.num_positions()
    lb = plant.GetPositionLowerLimits()
    rational_lb = forward_kin.ComputeTValue(lb, q_star)
    ub = plant.GetPositionUpperLimits()
    rational_ub = forward_kin.ComputeTValue(ub, q_star)
    volume_of_unit_sphere = 4.0*np.pi/3.0
    E = Hyperellipsoid(np.eye(3)/ellipsoid_epsilon, point)
    best_volume = ellipsoid_epsilon**dim * volume_of_unit_sphere
    
    link_poses_by_body_index_rat_pose = forward_kin.CalcLinkPoses(q_star, 
                                                         plant.world_body().index())
    X_WA_list = convert_RationalForwardPoseList_to_TransformExpressionList(link_poses_by_body_index_rat_pose)
    X_WB_list = convert_RationalForwardPoseList_to_TransformExpressionList(link_poses_by_body_index_rat_pose)

    inspector = query.inspector()
    pairs = inspector.GetCollisionCandidates()

    P = HPolyhedron.MakeBox(rational_lb, rational_ub)
    A = np.vstack((P.A(), np.zeros((10*len(pairs),3))))  # allow up to 10 faces per pair.
    b = np.concatenate((P.b(), np.zeros(10*len(pairs))))

    geom_ids = inspector.GetGeometryIds(GeometrySet(inspector.GetAllGeometryIds()), Role.kProximity)
    sets = {geom:MakeFromSceneGraph(query, geom, inspector.GetFrameId(geom)) for geom in geom_ids}
    body_indexes_by_geom_id = {geom:
                               plant.GetBodyFromFrameId(inspector.GetFrameId(geom)).index() for geom in geom_ids} 
    
    #Turn onto true to certify regions using SOS at each iteration.
    certify = False
    # refine polytopes if not certified collision free
    refine = False and certify
        
    iteration = 0
    num_faces = 2*len(lb)
    while True:
        ## Find separating hyperplanes

        for geomA, geomB in pairs:
            print(f"geomA={inspector.GetName(geomA)}, geomB={inspector.GetName(geomB)}")
            # Run snopt at the beginning
            while True:
                X_WA = X_WA_list[int(body_indexes_by_geom_id[geomA])]
                X_WB = X_WB_list[int(body_indexes_by_geom_id[geomB])]
                success, growth, qstar = GrowthVolumeRational(E,
                    X_WA, X_WB,
                    sets[geomA], sets[geomB], A[:num_faces,:], b[:num_faces] - dReal_polytope_tol, point)
                if success:
                    print(f"snopt_example={qstar}, growth = {growth}")
                    # Add a face to the polytope
                    A[num_faces,:], b[num_faces] = E.TangentPlane(qstar)
                    num_faces += 1
                else:
                    break

            if certify:
                pass
            

        if any([np.any(A[:num_faces,:] @ t > b[:num_faces]) for t in require_containment_points]):
            print("terminating because a required containment point would have not been contained")
            break

        P = HPolyhedron(A[:num_faces,:],b[:num_faces])

        E = P.MaximumVolumeInscribedEllipsoid()
        print(iteration)

        iteration += 1
        if iteration >= iteration_limit:
            break

        volume = volume_of_unit_sphere / np.linalg.det(E.A())
        if volume - best_volume <= termination_threshold:
            break
        best_volume = volume

    return P

In [17]:
#using hand-crafted seed points

seed_points = convert_q_to_t(np.array([[0.0, 0, 0], # startpoint
                        [0.8, -0.8, 1.3],  # blue low green up
                        [0.1, -1.2, 0.9],     # green low other up
                        [0.2, -0.6, 1.6],
                        [-0.5, -1.0, 1.9]]) ) # passing


iris_options = IrisOptions()
iris_options.require_sample_point_is_contained = True
iris_options.iteration_limit = 50
iris_options.enable_ibex = False

regions = []
ellipses = []
for i in range(seed_points.shape[0]):
    start_time = time.time()
#     hpoly = IrisInConfigurationSpace(plant, plant_context, seed_points[i,:], iris_options)
    hpoly = iris_rational_space(query, seed_points[i,:], require_containment_points=[seed_points[i,:]], iteration_limit=100)

    ellipse = hpoly.MaximumVolumeInscribedEllipsoid()
    print("Time: %6.2f \tVolume: %6.2f \tCenter:" % (time.time() - start_time, ellipse.Volume()),
          ellipse.center(), flush=True)
    ellipses.append(ellipse)
    regions.append(hpoly)


geomA=planar2dof_robot::link2_Collision@0000560e0ee81098, geomB=box_scene::box3_Collision@0000560e0ee81368
geomA=planar2dof_robot::link1_Collision@0000560e0ee80a08, geomB=box_scene::box2_Collision@0000560e0ee80eb8
snopt_example=[-0.30976193  0.          0.        ], growth = 9.595245239517713
geomA=planar2dof_robot::linka_Collision@0000560e0ee7a1a8, geomB=box_scene::box1_Collision@0000560e0ee80a08
geomA=planar2dof_robot::linka_Collision@0000560e0ee7a1a8, geomB=box_scene::box4_Collision@0000560e0ee7c648
snopt_example=[0.         0.         0.04181236], growth = 0.1748273122089364
geomA=planar2dof_robot::link2_Collision@0000560e0ee81098, geomB=box_scene::box2_Collision@0000560e0ee80eb8
snopt_example=[-0.11016135 -0.0513684  -0.05818764], growth = 1.8160038446954825
geomA=planar2dof_robot::link1_Collision@0000560e0ee80a08, geomB=box_scene::box1_Collision@0000560e0ee80a08
snopt_example=[ 0.61829463  0.         -0.05818764], growth = 38.56740476879112
geomA=planar2dof_robot::link1_Collision

In [15]:
def do_iris(q_seed, iris_options):
    start_time = time.time()
#     hpoly = IrisInConfigurationSpace(plant, plant_context, q_seed, iris_options)
    hpoly = iris_rational_space(query, q_seed, require_containment_points=[q_seed], iteration_limit=100)
  
    ellipse = hpoly.MaximumVolumeInscribedEllipsoid()
    print("Time: %6.2f \tVolume: %6.2f \tCenter:" % (time.time() - start_time, ellipse.Volume()),
          ellipse.center(), flush=True)
    return ellipse, hpoly

q_low_np = np.array(q_low)
q_high_np = np.array(q_high)

start = np.array([0.2, -1.6, 1.5])
target = np.array([0.8,-0.9,1.5])

do_reject_sampling = False
if do_reject_sampling:
    iris_options = IrisOptions()
    iris_options.require_sample_point_is_contained = True
    iris_options.iteration_limit = 50
    iris_options.enable_ibex = False

    regions = []
    ellipses = []
    its = 6
    seed_points = [start, target]

    for point in seed_points:
        ell, reg = do_iris(point, iris_options)
        regions.append(reg)
        ellipses.append(ell)
        
    for _ in range(its):
        #rejection sampling to get initial feasible point 
        found = False
        while not found:
            t = np.random.rand(3)
            q_samp = (1-t)*q_low_np + t*q_high_np
            found = (col_func_handle(*q_samp)==0.0)
        print("point found: ", q_samp)
        ell, reg = do_iris(q_samp, iris_options)
        if ell.Volume() < 10.0:
            regions.append(reg)
            ellipses.append(ell)
    

geomA=planar2dof_robot::link2_Collision@0000560e0ee81098, geomB=box_scene::box3_Collision@0000560e0ee81368
geomA=planar2dof_robot::link1_Collision@0000560e0ee80a08, geomB=box_scene::box2_Collision@0000560e0ee80eb8
snopt_example=[-0.30976193 -1.45740772  1.03833271], growth = 49.33264644884122
geomA=planar2dof_robot::linka_Collision@0000560e0ee7a1a8, geomB=box_scene::box1_Collision@0000560e0ee80a08
geomA=planar2dof_robot::linka_Collision@0000560e0ee7a1a8, geomB=box_scene::box4_Collision@0000560e0ee7c648
snopt_example=[ 0.2        -1.45740772  1.03833271], growth = 23.346924066310635
geomA=planar2dof_robot::link2_Collision@0000560e0ee81098, geomB=box_scene::box2_Collision@0000560e0ee80eb8
geomA=planar2dof_robot::link1_Collision@0000560e0ee80a08, geomB=box_scene::box1_Collision@0000560e0ee80a08
geomA=planar2dof_robot::link1_Collision@0000560e0ee80a08, geomB=planar2dof_robot::linka_Collision@0000560e0ee7a1a8
geomA=planar2dof_robot::link2_Collision@0000560e0ee81098, geomB=box_scene::box1_Co

In [18]:
def inpolycheck(q0,q1,q2, A, b):
    q = np.array([q0, q1, q2])
    res = np.min(1.0*(A@q-b<=0))
    #print(res)
    return res

meshes = []
col1 = 0x002BFF
col2 = 0x3EFF00 
idx = 0
step = 1/(1.0*len(regions))
R = np.array([[1, 0, 0],[0, 0, 1],[0,1, 0]])

for region in regions:
    A = region.A()
    b = region.b()
    col_hand = partial(inpolycheck, A=A, b=b)
    vertices, triangles = mcubes.marching_cubes_func(tuple(q_low), tuple(q_high), 50, 50, 50, col_hand, 0.5)
    regstr = "regions" +str(idx)
    ellstr = "ellipse" +str(idx)
    mat = meshcat.geometry.MeshLambertMaterial(color= int((1-idx*step)*col1 + idx*step*col2), wireframe=True)
    mat.opacity = 0.3
#     vis2['regions'][regstr].set_object(
#             meshcat.geometry.TriangularMeshGeometry((R@vertices.T).T, triangles),
#             mat)
    vis2['regions'][regstr].set_object(
            meshcat.geometry.TriangularMeshGeometry(vertices, triangles),
            mat)
    
    C = ellipses[idx].A()
    d = ellipses[idx].center()
    radii, R = np.linalg.eig(C.T@C)
    R[:,0] = R[:,0]*np.linalg.det(R)
    Rot = RotationMatrix(R)
 
    transf = RigidTransform(Rot, d)
    mat = meshcat.geometry.MeshLambertMaterial(color= int((1-idx*step)*col1 + idx*step*col2), wireframe=True)
    mat.opacity = 0.15
    vis2['ellipses'][ellstr].set_object(
            meshcat.geometry.Ellipsoid(np.divide(1,np.sqrt(radii))),
            meshcat.geometry.MeshLambertMaterial(color= int((1-idx*step)*col1 + idx*step*col2), wireframe=True))

    vis2['ellipses'][ellstr].set_transform(transf.GetAsMatrix4())
    
    idx+=1

In [15]:
# Solve path planning
solve_path_planning = False
if solve_path_planning:
    start = np.array([0.2, -1.6, 1.5])
    target = np.array([0.8,-0.9,1.5])
    start_time = time.time()
    spp = BsplineTrajectoryThroughUnionOfHPolyhedra(start, target, regions)
    spp.set_max_velocity([.8, .8, .8])
    spp.set_extra_control_points_per_region(3)

    # print(spp.num_regions())
    traj = spp.Solve()
    print(time.time() - start_time)
    print(traj.start_time())
    print(traj.end_time())
    #visualize
    maxit = 60
    pts = []
    for it in range(maxit):
        pts.append(traj.value(it*traj.end_time()/maxit))
        mat = meshcat.geometry.MeshLambertMaterial(color=0xFFF812)
        mat.reflectivity = 1.0
        vis2['traj']['points' + str(it)].set_object(
                    meshcat.geometry.Sphere(0.06), mat)
        vis2['traj']['points' + str(it)].set_transform(
                    meshcat.transformations.translation_matrix(pts[-1].reshape(-1,)))
        
        set_joint_angles(pts[-1].reshape(-1,))
        tf_l2 = plant.EvalBodyPoseInWorld(plant_context, plant.get_body(pydrake.multibody.tree.BodyIndex(3)))
        R_l2 = tf_l2.rotation()
        tl_l2 = R_l2@np.array([0,0,0.9]) + tf_l2.translation()
        
        tf_la = plant.EvalBodyPoseInWorld(plant_context, plant.get_body(pydrake.multibody.tree.BodyIndex(4)))
        R_la = tf_la.rotation()
        tl_la = R_la@np.array([0,0,1.2]) + tf_la.translation()
        
        
        mat = meshcat.geometry.MeshLambertMaterial(color=0x0029F1)
        mat.reflectivity = 1.0
        vis['traj']['link2']['points' + str(it)].set_object(
                    meshcat.geometry.Sphere(0.02), mat)
        vis['traj']['link2']['points' + str(it)].set_transform(
                    meshcat.transformations.translation_matrix(tl_l2))
        mat = meshcat.geometry.MeshLambertMaterial(color=0x07F100)
        mat.reflectivity = 1.0
        vis['traj']['linka']['points' + str(it)].set_object(
                    meshcat.geometry.Sphere(0.02), mat)
        vis['traj']['linka']['points' + str(it)].set_transform(
                    meshcat.transformations.translation_matrix(tl_la))
    

In [16]:
print(traj)

NameError: name 'traj' is not defined

In [17]:
#loop
idx = 0
going_fwd = True
steps = 1000
time_points = np.linspace(0, traj.end_time(), steps) 

while True:
    q = traj.value(time_points[idx])
    showres(q.reshape(-1,))
    if going_fwd:
        if idx + 1 < steps:
            idx += 1
        else:
            going_fwd = False
            idx -=1
    else:
        if idx-1 >= 0:
            idx -=1
        else:
            going_fwd = True
            idx +=1
            

NameError: name 'traj' is not defined

# CERTIFY REGIONS


In [18]:
def convert_Expression_Vect_to_Rational(vect):
    ret = vect.copy()
    for i, v in enumerate(vect):
        ret[i] = symHelpers.RationalFunctionFromExpression(v)
    return np.array(ret)

def ExprVectIsAllConstant(vect):
    for v in vect:
        if v.get_kind() != sym.ExpressionKind.Constant:
            return False
    return True

def RatVectIsAllConstant(vect):
    for v in vect:
        tmp = v.ToExpression()
        if tmp.get_kind() != sym.ExpressionKind.Constant:
            return False
    return True
    
def RationalVectHasCommonDenominatorAcrossVect(vect):
    ret = True
    for i in range(vect.shape[0]-1):
        tmp1 = vect[i].denominator()
        tmp2 = vect[i+1].denominator()
        ret = ret and tmp1.CoefficientsAlmostEqual(tmp2, 1e-10)
        ret = tmp1.EqualTo(tmp2)
    return ret

In [19]:

def polynomialHyperplaneConstraintPolynomial(t, leq_or_geq, rat_vertex, rat_base_point, plane):
    vect = rat_vertex - rat_base_point
    if not RationalVectHasCommonDenominatorAcrossVect(vect):
        raise ValueError("vertex needs to have same denominator across entries")
    numerator_vect = np.array([v.numerator() for v in vect])
    prod = 0
    for i in range(numerator_vect.shape[0]):
        prod += numerator_vect[i]*plane[i]
    if leq_or_geq == 'leq':
        poly_sos_expr = -prod+vect[0].denominator()
    elif leq_or_geq == 'geq':
        poly_sos_expr = prod-vect[0].denominator()
    else:
        raise ValueError('leq_or_geq must be leq or geq not {}'.format(leq_or_geq))
    poly_sos_expr.SetIndeterminates(sym.Variables(t))
    return poly_sos_expr

def polynomialHyperplaneConstraint(prog, plane, leq_or_geq, rat_vertex, rat_base_point, t,  
                                  t_Polyhedron_Constraint, lagrange_mult_degree = 4):
    poly_sos_expr = polynomialHyperplaneConstraintPolynomial(t, leq_or_geq, rat_vertex, rat_base_point, plane)
    cone_rhs, _ = prog.NewSosPolynomial(sym.Variables(t), lagrange_mult_degree)
    A = t_Polyhedron_Constraint.A()
    b = t_Polyhedron_Constraint.b()
    for i in range(A.shape[0]):
        gi = sym.Polynomial(A[i,:]@t-b[i])
        lagrange_mult_poly, Gram_matrix = prog.NewSosPolynomial(sym.Variables(t), lagrange_mult_degree)
        cone_rhs += lagrange_mult_poly*gi
    prog.AddConstraint(poly_sos_expr == cone_rhs)
    return prog
    
def constantHyperplaneConstraint(prog, plane, leq_or_geq, vertex, base_point):
    vect = vertex - base_point
    if leq_or_geq == 'leq':
        prog.AddLinearConstraint(plane@vect <= 1)
    elif leq_or_geq == 'geq':
        prog.AddLinearConstraint(plane@vect >= 1)
    else:
        raise ValueError('leq_or_geq must be leq or geq not {}'.format(leq_or_geq))
    return prog

def addHyperplaneConstraint(prog, plane, leq_or_geq, vertex, base_point, t, t_Polyhedron_Constraint, lagrange_mult_degree = 4):
    if ExprVectIsAllConstant(vertex) and ExprVectIsAllConstant(base_point):
        prog = constantHyperplaneConstraint(prog, plane, leq_or_geq, vertex, base_point)
    else:
        rat_vertex = convert_Expression_Vect_to_Rational(vertex)
        rat_base_point = convert_Expression_Vect_to_Rational(base_point)
        prog = polynomialHyperplaneConstraint(prog, plane, leq_or_geq, rat_vertex, rat_base_point, t,  
                                  t_Polyhedron_Constraint, lagrange_mult_degree)
                                
    return prog

def addSeparatingHyperplaneConstraint(prog, vertex1, vertex2, base_point, t, t_Polyhedron_Constraint, lagrange_mult_degree = 4, plane_name = None):
    if plane_name is not None:
        plane = prog.NewContinuousVariables(vertex1.shape[0], plane_name)
    else:
        plane = prog.NewContinuousVariables(vertex1.shape[0])
    prog = addHyperplaneConstraint(prog, plane, 'leq', vertex1, base_point, t, t_Polyhedron_Constraint, lagrange_mult_degree)
    prog = addHyperplaneConstraint(prog, plane, 'geq', vertex2, base_point, t, t_Polyhedron_Constraint, lagrange_mult_degree)
    return prog, plane


In [20]:
def cert_safe_poly_two_body(t, poly_to_cert, X_WA, VSetA, X_WB, VSetB, lagrange_mult_degree= 6):
    vertex1 = X_WA@RigidTransform(RotationMatrix(), VSetA.vertices()[:,0]).translation()
    if ExprVectIsAllConstant(vertex1):
        base_point = X_WA@RigidTransform(RotationMatrix(), VSetA.vertices().mean(axis = 1)).translation()
        order = (0,1)
    else:
        base_point = X_WB@RigidTransform(RotationMatrix(), VSetB.vertices().mean(axis = 1)).translation()
        order = (1,0)
    body_tup = VSetA, VSetB
    safe = True
    for i in range(VSetA.vertices().shape[1]):
        vertexA =X_WA@RigidTransform(RotationMatrix(), VSetA.vertices()[:,i]).translation()
        for j in range(VSetB.vertices().shape[1]):
            vertexB = X_WB@RigidTransform(RotationMatrix(), VSetB.vertices()[:,0]).translation()

            vert_tup = vertexA, vertexB
            vertex1, vertex2 =  vert_tup[order[0]], vert_tup[order[1]]
            prog = MathematicalProgram()

            prog, plane = addSeparatingHyperplaneConstraint(prog, vertex2, vertex1, base_point, t, poly_to_cert, lagrange_mult_degree)
            result = convSolver.Solve(prog)
            safe = safe and result.is_success()
            if not safe:
                return safe
            # print(result.GetSolution(plane))

    return safe

[<pydrake.geometry.optimization.HPolyhedron object at 0x7f8e03e8a0b0>, <pydrake.geometry.optimization.HPolyhedron object at 0x7f8e04446530>, <pydrake.geometry.optimization.HPolyhedron object at 0x7f8e03e989b0>, <pydrake.geometry.optimization.HPolyhedron object at 0x7f8e03e9bb70>, <pydrake.geometry.optimization.HPolyhedron object at 0x7f8e03e339b0>]


In [41]:
link_poses_by_body_index_rat_pose = forward_kin.CalcLinkPoses(q_star, 
                                                         plant.world_body().index())
X_WA_list = convert_RationalForwardPoseList_to_TransformExpressionList(link_poses_by_body_index_rat_pose)
X_WB_list = convert_RationalForwardPoseList_to_TransformExpressionList(link_poses_by_body_index_rat_pose)

inspector = query.inspector()
pairs = inspector.GetCollisionCandidates()


geom_ids = inspector.GetGeometryIds(GeometrySet(inspector.GetAllGeometryIds()), Role.kProximity)
HPolyhedronSets = {geom:MakeFromHPolyhedronOrEllipseSceneGraph(query, geom, inspector.GetFrameId(geom)) for geom in geom_ids}
VPolyhedronSets = {geom:MakeFromVPolytopeOrEllipseSceneGraph(query, geom, inspector.GetFrameId(geom)) for geom in geom_ids}

convSolver = MosekSolver()


body_indexes_by_geom_id = {geom:
                           plant.GetBodyFromFrameId(inspector.GetFrameId(geom)).index() for geom in geom_ids} 
kept_regions = []
for i, poly_to_cert in enumerate(regions):
    safe = True
    for geomA, geomB in pairs:
        X_WA = X_WA_list[int(body_indexes_by_geom_id[geomA])]
        X_WB = X_WB_list[int(body_indexes_by_geom_id[geomB])]
        if ExprVectIsAllConstant(X_WA.translation()) or ExprVectIsAllConstant(X_WB.translation()):
            bodyA, bodyB = VPolyhedronSets[geomA], VPolyhedronSets[geomB]
            safe = safe and  cert_safe_poly_two_body(t, poly_to_cert, X_WA, bodyA, X_WB, bodyB, lagrange_mult_degree=2)
        if not safe:
            continue
    if safe:
        kept_regions += [poly_to_cert]
    print("Completed {}/{} regions".format(i+1,len(regions)))
    print("Kept {}/{}".format(len(kept_regions), i+1))
    print()

Completed 1/5 regions
Kept 1/1

Completed 2/5 regions
Kept 2/2

Completed 3/5 regions
Kept 3/3

Completed 4/5 regions
Kept 4/4

Completed 5/5 regions
Kept 5/5



In [39]:
print([len(regions[i].b()) for i in range(len(regions))])

[14, 9, 9, 9, 6]
